In [101]:
# Cell 1.1: Install Libraries
!pip install streamlit pyngrok pandas

In [102]:
# Cell 1.2: Upload Dataset Files
from google.colab import files
import io
import pandas as pd
import sqlite3
from datetime import datetime

print("Please upload the following four CSV files:")
print("- food_listings_data.csv")
print("- claims_data.csv")
print("- receivers_data.csv")
print("- providers_data.csv")

uploaded = files.upload()

# Load uploaded files into pandas DataFrames
try:
    df_food_listings = pd.read_csv(io.BytesIO(uploaded['food_listings_data.csv']))
    df_claims = pd.read_csv(io.BytesIO(uploaded['claims_data.csv']))
    df_receivers = pd.read_csv(io.BytesIO(uploaded['receivers_data.csv']))
    df_providers = pd.read_csv(io.BytesIO(uploaded['providers_data.csv']))

    print("\nAll datasets loaded successfully!")
except KeyError as e:
    print(f"Error: Missing uploaded file - {e}. Please ensure all four files are uploaded.")
    # Exit or handle the error gracefully if files are missing
except Exception as e:
    print(f"An unexpected error occurred while loading files: {e}")

# Display first few rows to confirm data loading
print("\nFood Listings Data (first 5 rows):")
print(df_food_listings.head())
print("\nClaims Data (first 5 rows):")
print(df_claims.head())
print("\nReceivers Data (first 5 rows):")
print(df_receivers.head())
print("\nProviders Data (first 5 rows):")
print(df_providers.head())

Please upload the following four CSV files:
- food_listings_data.csv
- claims_data.csv
- receivers_data.csv
- providers_data.csv


Saving claims_data.csv to claims_data (3).csv
Saving food_listings_data.csv to food_listings_data (3).csv
Saving providers_data.csv to providers_data (3).csv
Saving receivers_data.csv to receivers_data (3).csv
Error: Missing uploaded file - 'food_listings_data.csv'. Please ensure all four files are uploaded.

Food Listings Data (first 5 rows):
   Food_ID Food_Name  Quantity Expiry_Date  Provider_ID     Provider_Type  \
0        1     Bread        43  2025-03-17          110     Grocery Store   
1        2      Soup        22  2025-03-24          791     Grocery Store   
2        3    Fruits        46  2025-03-28          478  Catering Service   
3        4    Fruits        15  2025-03-16          930        Restaurant   
4        5      Soup        14  2025-03-19          279        Restaurant   

           Location       Food_Type  Meal_Type  
0  South Kellyville  Non-Vegetarian  Breakfast  
1        West James  Non-Vegetarian     Dinner  
2       Lake Regina           Vegan  Breakfa

In [103]:
# Cell 1.3: Data Cleaning and Type Conversion

# Convert Expiry_Date in df_food_listings to datetime
df_food_listings['Expiry_Date'] = pd.to_datetime(df_food_listings['Expiry_Date'], format='%m/%d/%Y')

# Convert Timestamp in df_claims to datetime
df_claims['Timestamp'] = pd.to_datetime(df_claims['Timestamp'], format='%m/%d/%Y %H:%M')

print("\nData types converted successfully!")
print("\nFood Listings Data Info:")
df_food_listings.info()
print("\nClaims Data Info:")
df_claims.info()


Data types converted successfully!

Food Listings Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Food_ID        1000 non-null   int64         
 1   Food_Name      1000 non-null   object        
 2   Quantity       1000 non-null   int64         
 3   Expiry_Date    1000 non-null   datetime64[ns]
 4   Provider_ID    1000 non-null   int64         
 5   Provider_Type  1000 non-null   object        
 6   Location       1000 non-null   object        
 7   Food_Type      1000 non-null   object        
 8   Meal_Type      1000 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 70.4+ KB

Claims Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            -

In [104]:
# Cell 1.4: Create SQLite Database and Tables
# Connect to an SQLite database (it will be created if it doesn't exist)
conn = sqlite3.connect('food_waste_management.db')
cursor = conn.cursor()

# Define table schemas and create tables
# Providers Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Providers (
    Provider_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    Address TEXT,
    City TEXT,
    Contact TEXT
)
''')

# Receivers Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Receivers (
    Receiver_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    City TEXT,
    Contact TEXT
)
''')

# Food_Listings Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Food_Listings (
    Food_ID INTEGER PRIMARY KEY,
    Food_Name TEXT,
    Quantity INTEGER,
    Expiry_Date DATE,
    Provider_ID INTEGER,
    Provider_Type TEXT,
    Location TEXT,
    Food_Type TEXT,
    Meal_Type TEXT,
    FOREIGN KEY (Provider_ID) REFERENCES Providers(Provider_ID)
)
''')

# Claims Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Claims (
    Claim_ID INTEGER PRIMARY KEY,
    Food_ID INTEGER,
    Receiver_ID INTEGER,
    Status TEXT,
    Timestamp DATETIME,
    FOREIGN KEY (Food_ID) REFERENCES Food_Listings(Food_ID),
    FOREIGN KEY (Receiver_ID) REFERENCES Receivers(Receiver_ID)
)
''')

# Load data from pandas DataFrames into SQLite tables
df_providers.to_sql('Providers', conn, if_exists='replace', index=False)
df_receivers.to_sql('Receivers', conn, if_exists='replace', index=False)
df_food_listings.to_sql('Food_Listings', conn, if_exists='replace', index=False,
                        # SQLite does not have a native DATE type, use TEXT for dates,
                        # but pandas to_sql will handle datetime objects appropriately.
                        # We just need to ensure the format is consistent.
                        dtype={'Expiry_Date': 'TEXT'})
df_claims.to_sql('Claims', conn, if_exists='replace', index=False)

conn.commit()
print("Database 'food_waste_management.db' created and data loaded successfully!")

# You can close the connection now, or keep it open for queries
# For Streamlit app, we'll establish connection within the app.
# conn.close()

Database 'food_waste_management.db' created and data loaded successfully!


In [105]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('food_waste_management.db')
df = pd.read_sql_query("SELECT Food_ID, Food_Name, Quantity, Expiry_Date FROM Food_Listings", conn)
print("Total Food Listings in DB:", df.shape[0])
print(df.head())
conn.close()

Total Food Listings in DB: 1000
   Food_ID Food_Name  Quantity          Expiry_Date
0        1     Bread        43  2025-03-17 00:00:00
1        2      Soup        22  2025-03-24 00:00:00
2        3    Fruits        46  2025-03-28 00:00:00
3        4    Fruits        15  2025-03-16 00:00:00
4        5      Soup        14  2025-03-19 00:00:00


In [106]:
# Cell 2.1: Define SQL Query Execution Function

def run_sql_query(query):
    """Executes an SQL query and returns results as a pandas DataFrame."""
    conn = sqlite3.connect('food_waste_management.db') # Re-establish connection for each query
    try:
        df = pd.read_sql_query(query, conn)
        return df
    finally:
        conn.close()

print("SQL query execution function defined.")

SQL query execution function defined.


In [107]:
# Cell 2.2: Execute and Display SQL Queries

print("--- Executing SQL Queries for Analysis ---")

# Query 1: How many food providers and receivers are there in each city?
query1 = """
SELECT
    City,
    COUNT(DISTINCT Provider_ID) AS Num_Providers,
    COUNT(DISTINCT Receiver_ID) AS Num_Receivers
FROM (
    SELECT City, Provider_ID, NULL AS Receiver_ID FROM Providers
    UNION ALL
    SELECT City, NULL AS Provider_ID, Receiver_ID FROM Receivers
)
GROUP BY City
ORDER BY Num_Providers DESC, Num_Receivers DESC;
"""
print("\n1. Number of food providers and receivers in each city:")
df1 = run_sql_query(query1)
print(df1.to_string())

# Query 2: Which type of food provider (restaurant, grocery store, etc.) contributes the most food?
query2 = """
SELECT
    p.Type AS Provider_Type,
    SUM(fl.Quantity) AS Total_Food_Quantity
FROM
    Food_Listings fl
JOIN
    Providers p ON fl.Provider_ID = p.Provider_ID
GROUP BY
    p.Type
ORDER BY
    Total_Food_Quantity DESC;
"""
print("\n2. Food quantity contributed by each provider type:")
df2 = run_sql_query(query2)
print(df2.to_string())

# Query 3: What is the contact information of food providers in a specific city?
# Let's pick a city, e.g., 'New Jessica'
city_filter = 'New Jessica'
query3 = f"""
SELECT
    Name,
    Type,
    Address,
    Contact
FROM
    Providers
WHERE
    City = '{city_filter}';
"""
print(f"\n3. Contact information of food providers in {city_filter}:")
df3 = run_sql_query(query3)
print(df3.to_string())

# Query 4: Which receivers have claimed the most food?
query4 = """
SELECT
    r.Name AS Receiver_Name,
    SUM(fl.Quantity) AS Total_Food_Claimed
FROM
    Claims c
JOIN
    Food_Listings fl ON c.Food_ID = fl.Food_ID
JOIN
    Receivers r ON c.Receiver_ID = r.Receiver_ID
WHERE
    c.Status = 'Completed' -- Only count successfully claimed food
GROUP BY
    r.Name
ORDER BY
    Total_Food_Claimed DESC
LIMIT 10; -- Top 10 receivers
"""
print("\n4. Top 10 receivers by total food claimed (completed claims):")
df4 = run_sql_query(query4)
print(df4.to_string())

# Query 5: What is the total quantity of food available from all providers?
query5 = """
SELECT
    SUM(Quantity) AS Total_Available_Food
FROM
    Food_Listings;
"""
print("\n5. Total quantity of food available from all providers:")
df5 = run_sql_query(query5)
print(df5.to_string())

# Query 6: Which city has the highest number of food listings?
query6 = """
SELECT
    Location AS City,
    COUNT(Food_ID) AS Number_Of_Listings
FROM
    Food_Listings
GROUP BY
    Location
ORDER BY
    Number_Of_Listings DESC
LIMIT 10; -- Top 10 cities
"""
print("\n6. Top 10 cities with the highest number of food listings:")
df6 = run_sql_query(query6)
print(df6.to_string())

# Query 7: What are the most commonly available food types?
query7 = """
SELECT
    Food_Name,
    SUM(Quantity) AS Total_Quantity_Available
FROM
    Food_Listings
GROUP BY
    Food_Name
ORDER BY
    Total_Quantity_Available DESC
LIMIT 10; -- Top 10 food types
"""
print("\n7. Top 10 most commonly available food types:")
df7 = run_sql_query(query7)
print(df7.to_string())

# Query 8: How many food claims have been made for each food item?
query8 = """
SELECT
    fl.Food_Name,
    COUNT(c.Claim_ID) AS Total_Claims_Made
FROM
    Claims c
JOIN
    Food_Listings fl ON c.Food_ID = fl.Food_ID
GROUP BY
    fl.Food_Name
ORDER BY
    Total_Claims_Made DESC
LIMIT 10; -- Top 10 most claimed food items
"""
print("\n8. Top 10 food items with the most claims made:")
df8 = run_sql_query(query8)
print(df8.to_string())

# Query 9: Which provider has had the highest number of successful food claims?
query9 = """
SELECT
    p.Name AS Provider_Name,
    p.Type AS Provider_Type,
    COUNT(c.Claim_ID) AS Total_Successful_Claims
FROM
    Claims c
JOIN
    Food_Listings fl ON c.Food_ID = fl.Food_ID
JOIN
    Providers p ON fl.Provider_ID = p.Provider_ID
WHERE
    c.Status = 'Completed'
GROUP BY
    p.Name, p.Type
ORDER BY
    Total_Successful_Claims DESC
LIMIT 10; -- Top 10 providers
"""
print("\n9. Top 10 providers with the highest number of successful food claims:")
df9 = run_sql_query(query9)
print(df9.to_string())

# Query 10: What percentage of food claims are completed vs. pending vs. canceled?
query10 = """
SELECT
    Status,
    COUNT(Claim_ID) AS Num_Claims,
    CAST(COUNT(Claim_ID) AS REAL) * 100 / (SELECT COUNT(*) FROM Claims) AS Percentage
FROM
    Claims
GROUP BY
    Status
ORDER BY
    Percentage DESC;
"""
print("\n10. Percentage of food claims by status:")
df10 = run_sql_query(query10)
print(df10.to_string())

# Query 11: What is the average quantity of food claimed per receiver?
query11 = """
SELECT
    r.Name AS Receiver_Name,
    AVG(fl.Quantity) AS Average_Quantity_Per_Claim
FROM
    Claims c
JOIN
    Food_Listings fl ON c.Food_ID = fl.Food_ID
JOIN
    Receivers r ON c.Receiver_ID = r.Receiver_ID
WHERE
    c.Status = 'Completed' -- Consider only completed claims for meaningful average
GROUP BY
    r.Name
ORDER BY
    Average_Quantity_Per_Claim DESC
LIMIT 10; -- Top 10 receivers by avg quantity per claim
"""
print("\n11. Top 10 receivers by average quantity of food claimed per successful claim:")
df11 = run_sql_query(query11)
print(df11.to_string())

# Query 12: Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?
query12 = """
SELECT
    fl.Meal_Type,
    COUNT(c.Claim_ID) AS Total_Claims,
    SUM(CASE WHEN c.Status = 'Completed' THEN 1 ELSE 0 END) AS Completed_Claims,
    SUM(fl.Quantity) AS Total_Quantity_Claimed
FROM
    Claims c
JOIN
    Food_Listings fl ON c.Food_ID = fl.Food_ID
GROUP BY
    fl.Meal_Type
ORDER BY
    Total_Claims DESC;
"""
print("\n12. Claims and quantities by meal type:")
df12 = run_sql_query(query12)
print(df12.to_string())

# Query 13: What is the total quantity of food donated by each provider?
query13 = """
SELECT
    p.Name AS Provider_Name,
    p.Type AS Provider_Type,
    SUM(fl.Quantity) AS Total_Quantity_Donated
FROM
    Food_Listings fl
JOIN
    Providers p ON fl.Provider_ID = p.Provider_ID
GROUP BY
    p.Name, p.Type
ORDER BY
    Total_Quantity_Donated DESC
LIMIT 10; -- Top 10 donating providers
"""
print("\n13. Top 10 providers by total quantity of food donated:")
df13 = run_sql_query(query13)
print(df13.to_string())

# Additional Query 14 (CRUD): Find all food listings from a specific provider type that are expiring soon (e.g., within next 7 days from today)
# For 'today', we'll use a fixed date for reproducibility since Colab resets.
# Let's say "today" is '2025-03-15' for demonstration
today_date_str = '2025-03-15'
query14 = f"""
SELECT
    fl.Food_ID,
    fl.Food_Name,
    fl.Quantity,
    fl.Expiry_Date,
    fl.Provider_Type,
    fl.Location
FROM
    Food_Listings fl
WHERE
    fl.Expiry_Date BETWEEN '{today_date_str}' AND DATE('{today_date_str}', '+7 days')
ORDER BY
    fl.Expiry_Date ASC;
"""
print(f"\n14. Food listings expiring within 7 days from {today_date_str}:")
df14 = run_sql_query(query14)
print(df14.to_string())


# Additional Query 15 (CRUD): List Food Items available by a Provider and Receiver Location
# Let's filter by a specific Provider Type and Receiver City
provider_type_filter = 'Restaurant'
receiver_city_filter = 'Kellytown' # Assuming a receiver in this city might be interested

query15 = f"""
SELECT
    fl.Food_ID,
    fl.Food_Name,
    fl.Quantity,
    fl.Expiry_Date,
    fl.Provider_Type,
    p.Name AS Provider_Name,
    fl.Location AS Food_Location,
    fl.Food_Type,
    fl.Meal_Type
FROM
    Food_Listings fl
JOIN
    Providers p ON fl.Provider_ID = p.Provider_ID
WHERE
    fl.Provider_Type = '{provider_type_filter}' AND fl.Location = '{receiver_city_filter}'
ORDER BY
    fl.Expiry_Date ASC;
"""
print(f"\n15. Food listings by Provider Type '{provider_type_filter}' and Food Location '{receiver_city_filter}':")
df15 = run_sql_query(query15)
print(df15.to_string())

print("\n--- All SQL Queries Executed ---")

--- Executing SQL Queries for Analysis ---

1. Number of food providers and receivers in each city:
                          City  Num_Providers  Num_Receivers
0                    New Carol              3              0
1     South Christopherborough              3              0
2                 Lake Michael              2              1
3                   New Daniel              2              1
4               North Michelle              2              1
5                 Port Melissa              2              1
6             West Christopher              2              1
7                  Bradleyport              2              0
8                   Davidville              2              0
9                 East Anthony              2              0
10               East Jennifer              2              0
11                   East Lisa              2              0
12                East Melissa              2              0
13              East Stephanie              2 

In [164]:
# Cell 3.1: Create Streamlit App Script (app.py)

streamlit_app_lines = [
    "import streamlit as st",
    "import pandas as pd",
    "import sqlite3",
    "from datetime import datetime, timedelta",
    "import numpy as np",
    "",
    "# Function to get database connection - This is a connection factory, no caching here.",
    "def get_db_connection():",
    "    return sqlite3.connect('food_waste_management.db')",
    "",
    "# run_query function - Removed @st.cache_data here for fresh data retrieval on every run after rerun()",
    "def run_query(query, params=None):",
    "    conn = get_db_connection() ",
    "    try:",
    "        with st.spinner('Loading data...'):",
    "            if params is None:",
    "                df = pd.read_sql_query(query, conn)",
    "            else:",
    "                df = pd.read_sql_query(query, conn, params=params)",
    "        return df",
    "    finally:",
    "        conn.close() ",
    "",
    "# CRUD Operations Functions",
    "def add_food_listing(food_id, food_name, quantity, expiry_date, provider_id, provider_type, location, food_type, meal_type):",
    "    conn = sqlite3.connect('food_waste_management.db')",
    "    cursor = conn.cursor()",
    "    try:",
    "        cursor.execute(\"INSERT INTO Food_Listings (Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)\",",
    "                       (food_id, food_name, quantity, expiry_date, provider_id, provider_type, location, food_type, meal_type))",
    "        conn.commit()",
    "        st.success(f\"Food listing '{food_name}' added successfully! **Food ID: {food_id}**, Provider ID: {provider_id} 🎉\")",
    "        return True",
    "    except sqlite3.IntegrityError:",
    "        st.error(f\"Error: Food ID {food_id} already exists. Please choose a different ID. ⛔\")",
    "        return False",
    "    except Exception as e:",
    "        st.error(f\"An error occurred: {e} 🐛\")",
    "        return False",
    "    finally:",
    "        conn.close()",
    "",
    "def update_food_listing(food_id, new_quantity, new_expiry_date):",
    "    conn = sqlite3.connect('food_waste_management.db')",
    "    cursor = conn.cursor()",
    "    try:",
    "        cursor.execute(\"UPDATE Food_Listings SET Quantity = ?, Expiry_Date = ? WHERE Food_ID = ?\",",
    "                       (new_quantity, new_expiry_date, food_id))",
    "        conn.commit()",
    "        st.success(f\"Food listing ID {food_id} updated successfully! ✅\")",
    "    finally:",
    "        conn.close()",
    "",
    "def delete_food_listing(food_id):",
    "    conn = sqlite3.connect('food_waste_management.db')",
    "    cursor = conn.cursor()",
    "    try:",
    "        cursor.execute(\"DELETE FROM Claims WHERE Food_ID = ?\", (food_id,))",
    "        cursor.execute(\"DELETE FROM Food_Listings WHERE Food_ID = ?\", (food_id,))",
    "        conn.commit()",
    "        st.success(f\"Food listing ID {food_id} and related claims deleted successfully! 🗑️\")",
    "    finally:",
    "        conn.close()",
    "",
    "def create_claim(food_id, receiver_id, status, timestamp):",
    "    conn = sqlite3.connect('food_waste_management.db')",
    "    cursor = conn.cursor()",
    "    try:",
    "        food_check_query = \"SELECT Quantity FROM Food_Listings WHERE Food_ID = ?\"",
    "        current_food_details_df = pd.read_sql_query(food_check_query, conn, params=(food_id,))",
    "        if current_food_details_df.empty or current_food_details_df.iloc[0]['Quantity'] < 1:",
    "            st.error(f\"Food ID {food_id} is no longer available or quantity is 0. Claim failed. 😔\")",
    "            return False",
    "        current_qty = current_food_details_df.iloc[0]['Quantity']",
    "",
    "        expiry_dt_str = str(current_food_details_df.iloc[0]['Expiry_Date']) ",
    "        food_expiry_date = pd.to_datetime(expiry_dt_str, errors='coerce')",
    "        if pd.isna(food_expiry_date) or food_expiry_date.date() < datetime.now().date():",
    "            st.error(f\"Food ID {food_id} has expired ({food_expiry_date.strftime('%Y-%m-%d') if pd.notna(food_expiry_date) else 'N/A'}). Claim failed to prevent waste of already bad food. 🚫\")",
    "            return False",
    "",
    "        cursor.execute(\"INSERT INTO Claims (Food_ID, Receiver_ID, Status, Timestamp) VALUES (?, ?, ?, ?)\",",
    "                       (food_id, receiver_id, status, timestamp))",
    "        new_claim_id = cursor.lastrowid",
    "",
    "        new_quantity = max(0, current_qty - 1) ",
    "        cursor.execute(\"UPDATE Food_Listings SET Quantity = ? WHERE Food_ID = ?\", (new_quantity, food_id))",
    "",
    "        conn.commit()",
    "        st.success(f\"Claim for Food ID {food_id} by Receiver ID {receiver_id} created (Claim ID: {new_claim_id}) 🎉 Food quantity updated to {new_quantity}. \")",
    "        return True",
    "    except sqlite3.IntegrityError as e:",
    "        st.error(f\"Error creating claim: Check if Food ID {food_id} or Receiver ID {receiver_id} exist or claim already exists. Details: {e} ⛔\")",
    "        return False",
    "    except Exception as e:",
    "        st.error(f\"An unexpected error occurred during claim creation: {e} 🐛\")",
    "        return False",
    "    finally:",
    "        conn.close()",
    "",
    "def update_claim_status(claim_id, new_status, timestamp):",
    "    conn = sqlite3.connect('food_waste_management.db')",
    "    cursor = conn.cursor()",
    "    try:",
    "        cursor.execute(\"UPDATE Claims SET Status = ?, Timestamp = ? WHERE Claim_ID = ?\",",
    "                       (new_status, timestamp, claim_id))",
    "        conn.commit()",
    "        if cursor.rowcount > 0: ",
    "            st.success(f\"Claim ID {claim_id} status updated to '{new_status}' successfully! ✅\")",
    "        else:",
    "            st.warning(f\"Claim ID {claim_id} not found or status already '{new_status}'.\")",
    "        return True",
    "    except Exception as e:",
    "        st.error(f\"Error updating claim status: {e} 🐛\")",
    "        return False",
    "    finally:",
    "        conn.close()",
    "",
    "def delete_claim(claim_id):",
    "    conn = sqlite3.connect('food_waste_management.db')",
    "    cursor = conn.cursor()",
    "    try:",
    "        cursor.execute(\"DELETE FROM Claims WHERE Claim_ID = ?\", (claim_id,))",
    "        conn.commit()",
    "        if cursor.rowcount > 0: ",
    "            st.success(f\"Claim ID {claim_id} deleted successfully! 🗑️\")",
    "        else:",
    "            st.warning(f\"Claim ID {claim_id} not found.\")",
    "        return True",
    "    except Exception as e:",
    "        st.error(f\"Error deleting claim: {e} 🐛\")",
    "        return False",
    "    finally:",
    "        conn.close()",
    "",
    "def safe_int_conversion(value):",
    "    \"\"\"Converts a value to int safely, handling NaN/None.\"\"\"",
    "    if pd.isna(value) or value is None:",
    "        return None",
    "    try:",
    "        return int(float(value))",
    "    except (ValueError, TypeError):",
    "        return None",
    "",
    "def format_sql_query_for_display(query_template, params):",
    "    \"\"\"Replaces '?' placeholders in a query template with actual parameter values for display.\"\"\"",
    "    display_query = query_template",
    "    param_list = list(params) if params is not None else []",
    "    ",
    "    for p_val in param_list:",
    "        if isinstance(p_val, (int, float)):",
    "            display_query = display_query.replace(\"?\", str(p_val), 1)",
    "        else:",
    "            display_query = display_query.replace(\"?\", f\"'{p_val}'\", 1)",
    "    return display_query",
    "",
    "def convert_df_to_csv(df):",
    "    \"\"\"Converts a Pandas DataFrame to CSV for download.\"\"\"  ",
    "    return df.to_csv(index=False).encode('utf-8')",
    "",
    "# Function to apply expiry highlighting (for Food Listings and At-Risk Food pages)",
    "def highlight_expiry_bold(s):",
    "    today = datetime.now().date()",
    "    style_string = [''] * len(s) ",
    "    ",
    "    if 'Expiry_Date' in s.index and not pd.isna(s['Expiry_Date']):",
    "        expiry_date_val_dt = pd.to_datetime(s['Expiry_Date'], errors='coerce')",
    "        if pd.isna(expiry_date_val_dt): ",
    "            return [''] * len(s)",
    "        expiry_date_val = expiry_date_val_dt.date()",
    "        ",
    "        if expiry_date_val < today: ",
    "            style_string = ['background-color: #ffe6e6; color: #8B0000; font-weight: bold;'] * len(s) ",
    "        elif (expiry_date_val - today).days <= 3: ",
    "            style_string = ['background-color: #fffacd; color: #cc6600; font-weight: bold;'] * len(s) ",
    "    return style_string",
    "",
    "# --- Streamlit App Layout ---",
    "st.set_page_config(layout=\"wide\", page_title=\"Local Food Management App | Reducing Waste, Feeding Communities\")",
    "st.title(\"🍔 Local Food Wastage Management System 🍎\")",
    "",
    "st.sidebar.header(\"Navigation\")",
    "page_selection = st.sidebar.radio(\"Go to\", [\"Dashboard\", \"Analytical Queries\", \"Dataset Viewer\", \"Food Listings\", \"Claims & Status\", \"CRUD Operations\", \"At-Risk Food\"])",
    "",
    "if page_selection == \"Dashboard\":",
    "    st.header(\"Dashboard Overview \📊\")",
    "",
    "    st.markdown(\"\"\"",
    "    Welcome to the Local Food Wastage Management System! This platform connects surplus food from providers with individuals and organizations in need. Our goal is to reduce food waste and contribute to food security. Explore the sections to see available food, claims, and analytical insights. \\n",
    "    You can refresh data by clicking the 'Refresh Data' button if you've made recent changes. \\n",
    "    ***Data current as of:*** \\n"
    f"***{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}***"
    "\"\"\")",
    "",
    "    if st.button('Refresh Dashboard Data 🔄'):",
    "        st.rerun()",
    "",
    "    st.write('---')",
    "",
    "    col1, col2, col3 = st.columns(3)",
    "    with col1:",
    "        total_food_q = run_query(\"SELECT SUM(Quantity) FROM Food_Listings\").iloc[0,0]",
    "        st.metric(\"Total Food Available\", f\"{total_food_q if total_food_q else 0} units\")",
    "    with col2:",
    "        total_providers = run_query(\"SELECT COUNT(DISTINCT Provider_ID) FROM Providers\").iloc[0,0]",
    "        st.metric(\"Total Food Providers\", total_providers)",
    "    with col3:",
    "        total_receivers = run_query(\"SELECT COUNT(DISTINCT Receiver_ID) FROM Receivers\").iloc[0,0]",
    "        st.metric(\"Total Food Receivers\", total_receivers)",
    "",
    "    st.write('---')",
    "    st.subheader(\"Key Trends & Statistics \📈\")",
    "",
    "    provider_contrib_query = \"\"\"",
    "    SELECT p.Type AS Provider_Type, SUM(fl.Quantity) AS Total_Food_Quantity",
    "    FROM Food_Listings fl JOIN Providers p ON fl.Provider_ID = p.Provider_ID",
    "    GROUP BY p.Type ORDER BY Total_Food_Quantity DESC;",
    "    \"\"\"",
    "    provider_contrib_df = run_query(provider_contrib_query)",
    "    st.write(\"**Food Quantity Contributed by Each Provider Type:**\")",
    "    st.bar_chart(provider_contrib_df.set_index('Provider_Type'))",
    "",
    "    claims_status_query = \"\"\"",
    "    SELECT Status, COUNT(Claim_ID) AS Num_Claims, CAST(COUNT(Claim_ID) AS REAL) * 100 / (SELECT COUNT(*) FROM Claims) AS Percentage",
    "    FROM Claims GROUP BY Status ORDER BY Percentage DESC;",
    "    \"\"\"",
    "    claims_status_df = run_query(claims_status_query)",
    "    st.write(\"**Percentage of Food Claims by Status:**\")",
    "    st.bar_chart(claims_status_df.set_index('Status'))",
    "",
    "    col_dash_queries1, col_dash_queries2 = st.columns(2)",
    "    with col_dash_queries1:",
    "        total_donated_by_provider_query = \"\"\"",
    "        SELECT p.Name AS Provider_Name, p.Type AS Provider_Type, SUM(fl.Quantity) AS Total_Quantity_Donated",
    "        FROM Food_Listings fl JOIN Providers p ON fl.Provider_ID = p.Provider_ID",
    "        GROUP BY p.Name, p.Type ORDER BY Total_Quantity_Donated DESC LIMIT 10;",
    "        \"\"\"",
    "        total_donated_by_provider_df = run_query(total_donated_by_provider_query)",
    "        st.write(\"**Top 10 Providers by Total Quantity Donated:**\")",
    "        st.dataframe(total_donated_by_provider_df, use_container_width=True)",
    "        st.download_button(label='Download Top Providers (CSV) 📥', data=convert_df_to_csv(total_donated_by_provider_df), file_name='top_donating_providers.csv', mime='text/csv')",
    "",
    "    with col_dash_queries2:",
    "        top_receivers_query = \"\"\"",
    "        SELECT r.Name AS Receiver_Name, SUM(fl.Quantity) AS Total_Food_Claimed",
    "        FROM Claims c JOIN Food_Listings fl ON c.Food_ID = fl.Food_ID JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID",
    "        WHERE c.Status = 'Completed' GROUP BY r.Name ORDER BY Total_Food_Claimed DESC LIMIT 10;",
    "        \"\"\"",
    "        top_receivers_df = run_query(top_receivers_query)",
    "        st.write(\"**Top 10 Receivers by Food Claimed (Completed):**\")",
    "        st.dataframe(top_receivers_df, use_container_width=True)",
    "        st.download_button(label='Download Top Receivers (CSV) 📥', data=convert_df_to_csv(top_receivers_df), file_name='top_claiming_receivers.csv', mime='text/csv')",
    "",
    "elif page_selection == \"Analytical Queries\":",
    "    st.header(\"Full Analytical Queries \🔬\")",
    "    st.markdown(\"Here you can view the detailed results for all analytical questions defined in the project. Each section displays the result table and its corresponding SQL query.\")",
    "",
    "    st.write('---')",
    "    st.subheader(\"1. Food Providers & Receivers Insights: City Counts \🌆\")",
    "    query1 = \"\"\"",
    "    SELECT City, COUNT(DISTINCT Provider_ID) AS Num_Providers, COUNT(DISTINCT Receiver_ID) AS Num_Receivers",
    "    FROM (SELECT City, Provider_ID, NULL AS Receiver_ID FROM Providers",
    "          UNION ALL SELECT City, NULL AS Provider_ID, Receiver_ID FROM Receivers) ",
    "    GROUP BY City ORDER BY Num_Providers DESC, Num_Receivers DESC;",
    "    \"\"\"",
    "    st.write(\"**Question:** How many food providers and receivers are there in each city?\")",
    "    st.dataframe(run_query(query1), use_container_width=True)",
    "    st.code(query1)",
    "",
    "    st.write('---')",
    "    st.subheader(\"2. Food Providers & Receivers Insights: Provider Contributions by Type \🍽️\")",
    "    query2_text = \"\"\"",
    "    SELECT p.Type AS Provider_Type, SUM(fl.Quantity) AS Total_Food_Quantity",
    "    FROM Food_Listings fl JOIN Providers p ON fl.Provider_ID = p.Provider_ID",
    "    GROUP BY p.Type ORDER BY Total_Food_Quantity DESC;",
    "    \"\"\"",
    "    st.write(\"**Question:** Which type of food provider (restaurant, grocery store, etc.) contributes the most food?\")",
    "    st.dataframe(run_query(query2_text), use_container_width=True)",
    "    st.code(query2_text)",
    "",
    "    st.write('---')",
    "    st.subheader(\"3. Food Providers & Receivers Insights: Provider Contact Information \📞\")",
    "    provider_cities_q3 = run_query(\"SELECT DISTINCT City FROM Providers WHERE City IS NOT NULL\").iloc[:,0].tolist()",
    "    if not provider_cities_q3: provider_cities_q3 = ['(N/A)']",
    "    city_filter_q3 = st.selectbox(\"Select City for Query 3\", provider_cities_q3, key='q3_city_select')",
    "    query3_text = f\"\"\"",
    "    SELECT Name, Type, Address, Contact",
    "    FROM Providers",
    "    WHERE City = '{city_filter_q3}';",
    "    \"\"\"",
    "    st.write(f\"**Question:** What is the contact information of food providers in {city_filter_q3}?\")",
    "    st.dataframe(run_query(query3_text), use_container_width=True)",
    "    st.code(query3_text)",
    "",
    "    st.write('---')",
    "    st.subheader(\"4. Claims & Distribution Insights: Receivers Claiming Most Food \🎯\")",
    "    query4_text = \"\"\"",
    "    SELECT r.Name AS Receiver_Name, SUM(fl.Quantity) AS Total_Food_Claimed",
    "    FROM Claims c JOIN Food_Listings fl ON c.Food_ID = fl.Food_ID JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID",
    "    WHERE c.Status = 'Completed'",
    "    GROUP BY r.Name ORDER BY Total_Food_Claimed DESC LIMIT 10;",
    "    \"\"\"",
    "    st.write(\"**Question:** Which receivers have claimed the most food?\")",
    "    st.dataframe(run_query(query4_text), use_container_width=True)",
    "    st.code(query4_text)",
    "",
    "    st.write('---')",
    "    st.subheader(\"5. Food Listings & Availability: Total Food Available \📦\")",
    "    query5_text = \"\"\"",
    "    SELECT SUM(Quantity) AS Total_Available_Food FROM Food_Listings;",
    "    \"\"\"",
    "    st.write(\"**Question:** What is the total quantity of food available from all providers?\")",
    "    st.dataframe(run_query(query5_text), use_container_width=True)",
    "    st.code(query5_text)",
    "",
    "    st.write('---')",
    "    st.subheader(\"6. Food Listings & Availability: Cities with Most Food Listings \📍\")",
    "    query6_text = \"\"\"",
    "    SELECT Location AS City, COUNT(Food_ID) AS Number_Of_Listings",
    "    FROM Food_Listings GROUP BY Location ORDER BY Number_Of_Listings DESC LIMIT 10;",
    "    \"\"\"",
    "    st.write(\"**Question:** Which city has the highest number of food listings?\")",
    "    st.dataframe(run_query(query6_text), use_container_width=True)",
    "    st.code(query6_text)",
    "",
    "    st.write('---')",
    "    st.subheader(\"7. Food Listings & Availability: Most Commonly Available Food Types \🍎\")",
    "    query7_text = \"\"\"",
    "    SELECT Food_Name, SUM(Quantity) AS Total_Quantity_Available",
    "    FROM Food_Listings GROUP BY Food_Name ORDER BY Total_Quantity_Available DESC LIMIT 10;",
    "    \"\"\"",
    "    st.write(\"**Question:** What are the most commonly available food types?\")",
    "    st.dataframe(run_query(query7_text), use_container_width=True)",
    "    st.code(query7_text)",
    "",
    "    st.write('---')",
    "    st.subheader(\"8. Claims & Distribution: Claims per Food Item \📋\")",
    "    query8_text = \"\"\"",
    "    SELECT fl.Food_Name, COUNT(c.Claim_ID) AS Total_Claims_Made",
    "    FROM Claims c JOIN Food_Listings fl ON c.Food_ID = fl.Food_ID",
    "    GROUP BY fl.Food_Name ORDER BY Total_Claims_Made DESC LIMIT 10;",
    "    \"\"\"",
    "    st.write(\"**Question:** How many food claims have been made for each food item?\")",
    "    st.dataframe(run_query(query8_text), use_container_width=True)",
    "    st.code(query8_text)",
    "",
    "    st.write('---')",
    "    st.subheader(\"9. Claims & Distribution: Providers with Highest Successful Claims \🏆\")",
    "    query9_text = \"\"\"",
    "    SELECT p.Name AS Provider_Name, p.Type AS Provider_Type, COUNT(c.Claim_ID) AS Total_Successful_Claims",
    "    FROM Claims c JOIN Food_Listings fl ON c.Food_ID = fl.Food_ID JOIN Providers p ON fl.Provider_ID = p.Provider_ID",
    "    WHERE c.Status = 'Completed'",
    "    GROUP BY p.Name, p.Type ORDER BY Total_Successful_Claims DESC LIMIT 10;",
    "    \"\"\"",
    "    st.write(\"**Question:** Which provider has had the highest number of successful food claims?\")",
    "    st.dataframe(run_query(query9_text), use_container_width=True)",
    "    st.code(query9_text)",
    "",
    "    st.write('---')",
    "    st.subheader(\"10. Claims & Distribution: Claim Status Percentage \📊\")",
    "    query10_text = \"\"\"",
    "    SELECT Status, COUNT(Claim_ID) AS Num_Claims, CAST(COUNT(Claim_ID) AS REAL) * 100 / (SELECT COUNT(*) FROM Claims) AS Percentage",
    "    FROM Claims GROUP BY Status ORDER BY Percentage DESC;",
    "    \"\"\"",
    "    st.write(\"**Question:** What percentage of food claims are completed vs. pending vs. canceled?\")",
    "    st.dataframe(run_query(query10_text), use_container_width=True)",
    "    st.code(query10_text)",
    "",
    "    st.write('---')",
    "    st.subheader(\"11. Claims & Distribution: Average Quantity Claimed per Receiver \🛒\")",
    "    query11_text = \"\"\"",
    "    SELECT r.Name AS Receiver_Name, AVG(fl.Quantity) AS Average_Quantity_Per_Claim",
    "    FROM Claims c JOIN Food_Listings fl ON c.Food_ID = fl.Food_ID JOIN Receivers r ON c.Receiver_ID = r.Receiver_ID",
    "    WHERE c.Status = 'Completed'",
    "    GROUP BY r.Name ORDER BY Average_Quantity_Per_Claim DESC LIMIT 10;",
    "    \"\"\"",
    "    st.write(\"**Question:** What is the average quantity of food claimed per receiver?\")",
    "    st.dataframe(run_query(query11_text), use_container_width=True)",
    "    st.code(query11_text)",
    "",
    "    st.write('---')",
    "    st.subheader(\"12. Claims & Distribution: Most Claimed Meal Type \🍜\")",
    "    query12_text = \"\"\"",
    "    SELECT fl.Meal_Type, COUNT(c.Claim_ID) AS Total_Claims, SUM(CASE WHEN c.Status = 'Completed' THEN 1 ELSE 0 END) AS Completed_Claims, SUM(fl.Quantity) AS Total_Quantity_Claimed",
    "    FROM Claims c JOIN Food_Listings fl ON c.Food_ID = fl.Food_ID",
    "    GROUP BY fl.Meal_Type ORDER BY Total_Claims DESC;",
    "    \"\"\"",
    "    st.write(\"**Question:** Which meal type (breakfast, lunch, dinner, snacks) is claimed the most?\")",
    "    st.dataframe(run_query(query12_text), use_container_width=True)",
    "    st.code(query12_text)",
    "",
    "    st.write('---')",
    "    st.subheader(\"13. Analysis & Insights: Total Food Donated by Each Provider \🎁\")",
    "    query13_text = \"\"\"",
    "    SELECT p.Name AS Provider_Name, p.Type AS Provider_Type, SUM(fl.Quantity) AS Total_Quantity_Donated",
    "    FROM Food_Listings fl JOIN Providers p ON fl.Provider_ID = p.Provider_ID",
    "    GROUP BY p.Name, p.Type ORDER BY Total_Quantity_Donated DESC LIMIT 10;",
    "    \"\"\"",
    "    st.write(\"**Question:** What is the total quantity of food donated by each provider?\")",
    "    st.dataframe(run_query(query13_text), use_container_width=True)",
    "    st.code(query13_text)",
    "",
    "    st.write('---')",
    "    st.subheader(\"14. Analysis & Insights: Food Listings Expiring Soon (Custom Filter) \⏰\")",
    "    today_date_str_q14 = '2025-03-15' ",
    "    expiring_soon_days = st.slider(\"Days until expiry\", 1, 30, 7, key='q14_expiry_days', help=\"Adjust days to see items expiring within that timeframe.\")",
    "    query14_text = f\"\"\"",
    "    SELECT fl.Food_ID, fl.Food_Name, fl.Quantity, fl.Expiry_Date, fl.Provider_Type, fl.Location",
    "    FROM Food_Listings fl",
    "    WHERE date(fl.Expiry_Date) BETWEEN date('{today_date_str_q14}') AND date('{today_date_str_q14}', '+{expiring_soon_days} days')",
    "    ORDER BY fl.Expiry_Date ASC;",
    "    \"\"\"",
    "    st.write(f\"**Question:** Find food listings expiring within `{expiring_soon_days}` days from `{today_date_str_q14}`.\")",
    "    st.dataframe(run_query(query14_text), use_container_width=True)",
    "    st.code(format_sql_query_for_display(query14_text, [today_date_str_q14, today_date_str_q14, expiring_soon_days]))",
    "",
    "    st.write('---')",
    "    st.subheader(\"15. Analysis & Insights: Food by Provider Type & Location (Custom Filter) \🗺️\")",
    "    available_provider_types_q15 = run_query(\"SELECT DISTINCT Type FROM Providers WHERE Type IS NOT NULL\").iloc[:,0].tolist()",
    "    available_locations_q15 = run_query(\"SELECT DISTINCT Location FROM Food_Listings WHERE Location IS NOT NULL\").iloc[:,0].tolist()",
    "",
    "    if not available_provider_types_q15: available_provider_types_q15 = ['(N/A)']",
    "    if not available_locations_q15: available_locations_q15 = ['(N/A)']",
    "",
    "    selected_provider_type_q15 = st.selectbox(\"Select Provider Type\", available_provider_types_q15, key='q15_provider_type', help=\"Filter listings by provider type.\")",
    "    selected_food_location_q15 = st.selectbox(\"Select Food Location\", available_locations_q15, key='q15_location', help=\"Filter listings by food item's location.\")",
    "",
    "    query15 = f\"\"\"",
    "    SELECT fl.Food_ID, fl.Food_Name, fl.Quantity, fl.Expiry_Date, fl.Provider_Type, p.Name AS Provider_Name, fl.Location AS Food_Location, fl.Food_Type, fl.Meal_Type",
    "    FROM Food_Listings fl JOIN Providers p ON fl.Provider_ID = p.Provider_ID",
    "    WHERE fl.Provider_Type = '{selected_provider_type_q15}' AND fl.Location = '{selected_food_location_q15}'",
    "    ORDER BY fl.Expiry_Date ASC;",
    "    \"\"\"",
    "    st.write(f\"**Question:** List Food Items available from `{selected_provider_type_q15}` providers in `{selected_food_location_q15}`.\")",
    "    st.dataframe(run_query(query15), use_container_width=True)",
    "    st.code(format_sql_query_for_display(query15, [selected_provider_type_q15, selected_food_location_q15]))",
    "",
    "elif page_selection == \"Dataset Viewer\":",
    "    st.header(\"Raw Dataset Viewer \🔍\")",
    "    st.markdown(\"Browse the raw content of each table in the database.\")",
    "",
    "    if st.button('Refresh Datasets 🔄', key='refresh_datasets'):",
    "        st.rerun()",
    "",
    "    table_to_view = st.selectbox(\"Select Table to View\", [\"Food_Listings\", \"Claims\", \"Providers\", \"Receivers\"])",
    "",
    "    if table_to_view == \"Food_Listings\":",
    "        st.subheader(\"Food Listings Table \📜\")",
    "        df = run_query(\"SELECT * FROM Food_Listings\")",
    "        st.dataframe(df, use_container_width=True)",
    "        if not df.empty: ",
    "            food_type_counts = df['Food_Type'].value_counts().reset_index()",
    "            food_type_counts.columns = ['Food_Type', 'Count']",
    "            st.write(\"Food Type Distribution:\")",
    "            st.bar_chart(food_type_counts.set_index('Food_Type'))",
    "            st.download_button(label='Download Food Listings (CSV) 📥', data=convert_df_to_csv(df), file_name='food_listings.csv', mime='text/csv')",
    "",
    "    elif table_to_view == \"Claims\":",
    "        st.subheader(\"Claims Table \📄\")",
    "        df = run_query(\"SELECT * FROM Claims\")",
    "        st.dataframe(df, use_container_width=True)",
    "        if not df.empty: ",
    "            claim_status_counts = df['Status'].value_counts().reset_index()",
    "            claim_status_counts.columns = ['Status', 'Count']",
    "            st.write(\"Claim Status Distribution:\")",
    "            st.bar_chart(claim_status_counts.set_index('Status'))",
    "            st.download_button(label='Download Claims Data (CSV) 📥', data=convert_df_to_csv(df), file_name='claims_data.csv', mime='text/csv')",
    "",
    "    elif table_to_view == \"Providers\":",
    "        st.subheader(\"Providers Table \🏪\")",
    "        df = run_query(\"SELECT * FROM Providers\")",
    "        st.dataframe(df, use_container_width=True)",
    "        if not df.empty: ",
    "            provider_type_counts = df['Type'].value_counts().reset_index()",
    "            provider_type_counts.columns = ['Type', 'Count']",
    "            st.write(\"Provider Type Distribution:\")",
    "            st.bar_chart(provider_type_counts.set_index('Type'))",
    "            st.download_button(label='Download Providers Data (CSV) 📥', data=convert_df_to_csv(df), file_name='providers_data.csv', mime='text/csv')",
    "",
    "    elif table_to_view == \"Receivers\":",
    "        st.subheader(\"Receivers Table \👨‍👩‍👧‍👦\")",
    "        df = run_query(\"SELECT * FROM Receivers\")",
    "        st.dataframe(df, use_container_width=True)",
    "        if not df.empty: ",
    "            receiver_type_counts = df['Type'].value_counts().reset_index()",
    "            receiver_type_counts.columns = ['Type', 'Count']",
    "            st.write(\"Receiver Type Distribution:\")",
    "            st.bar_chart(receiver_type_counts.set_index('Type'))",
    "            st.download_button(label='Download Receivers Data (CSV) 📥', data=convert_df_to_csv(df), file_name='receivers_data.csv', mime='text/csv')",
    "",
    "elif page_selection == \"Food Listings\":",
    "    st.header(\"Available Food Listings \清单\")",
    "",
    "    if st.button('Refresh Listings 🔄', key='refresh_food_listings'):",
    "        st.rerun()",
    "",
    "    col_fl_filter, col_fl_content = st.columns([1, 3]) ",
    "    ",
    "    with col_fl_filter:",
    "        st.subheader(\"Filter Options\")",
    "        st.write('---')",
    "",
    "        all_locations = run_query(\"SELECT DISTINCT Location FROM Food_Listings WHERE Location IS NOT NULL\").iloc[:,0].tolist()",
    "        all_provider_types = run_query(\"SELECT DISTINCT Type FROM Providers WHERE Type IS NOT NULL\").iloc[:,0].tolist()",
    "        all_food_names = run_query(\"SELECT DISTINCT Food_Name FROM Food_Listings WHERE Food_Name IS NOT NULL\").iloc[:,0].tolist()",
    "        all_meal_types = run_query(\"SELECT DISTINCT Meal_Type FROM Food_Listings WHERE Meal_Type IS NOT NULL\").iloc[:,0].tolist()",
    "",
    "        if not all_locations: all_locations = ['(N/A)']",
    "        if not all_provider_types: all_provider_types = ['(N/A)']",
    "        if not all_food_names: all_food_names = ['(N/A)']",
    "        if not all_meal_types: all_meal_types = ['(N/A)']",
    "",
    "        search_keyword = st.text_input(\"Search by Keyword (Food Name or Location)\", \"\", help=\"Type to filter by Food Name or Location (case-insensitive). \").strip().lower()",
    "",
    "        selected_location = st.selectbox(\"Location\", [\"All\"] + all_locations, help=\"Filter listings by geographical location.\")",
    "        selected_provider_type = st.selectbox(\"Provider Type\", [\"All\"] + all_provider_types, help=\"Filter by the type of food provider (e.g., Restaurant, Grocery Store).\")",
    "        selected_food_type_name = st.selectbox(\"Food Item Name\", [\"All\"] + all_food_names, help=\"Filter by specific food item (e.g., Bread, Fruits, Chicken).\")",
    "        selected_meal_type = st.selectbox(\"Meal Type\", [\"All\"] + all_meal_types, help=\"Suggest suitable meal type for this food.\")",
    "",
    "        min_quantity = st.number_input(\"Minimum Quantity\", min_value=0, value=0, help=\"Show items with at least this many units available.\")",
    "        st.write(\"Expiry Date Range:\")",
    "        today_dt = datetime.now()",
    "        default_start_date = today_dt.date()",
    "        default_end_date = (today_dt + timedelta(days=30)).date()",
    "        expiry_start_date = st.date_input(\"From (Expiry Date)\", value=default_start_date, help=\"Start date for expiry range.\")",
    "        expiry_end_date = st.date_input(\"To (Expiry Date)\", value=default_end_date, help=\"End date for expiry range.\") ",
    "        ",
    "        st.write('---')",
    "",
    "    with col_fl_content:",
    "        st.subheader(\"Filtered Food Listings\")",
    "",
    "        query = \"\"\"",
    "        SELECT",
    "            fl.Food_ID,",
    "            fl.Food_Name,",
    "            fl.Quantity,",
    "            fl.Expiry_Date,",
    "            p.Name AS Provider_Name,",
    "            fl.Provider_Type,",
    "            fl.Location,",
    "            fl.Food_Type,",
    "            fl.Meal_Type,",
    "            p.Contact AS Provider_Contact",
    "        FROM",
    "            Food_Listings fl",
    "        JOIN",
    "            Providers p ON fl.Provider_ID = p.Provider_ID",
    "        WHERE",
    "            fl.Quantity >= ?",
    "        \"\"\"",
    "        params = [min_quantity]",
    "",
    "        if selected_location != \"All\":",
    "            query += \" AND fl.Location = ?\"",
    "            params.append(selected_location)",
    "        if selected_provider_type != \"All\":",
    "            query += \" AND fl.Provider_Type = ?\"",
    "            params.append(selected_provider_type)",
    "        if selected_food_type_name != \"All\":",
    "            query += \" AND fl.Food_Name = ?\"",
    "            params.append(selected_food_type_name)",
    "        if selected_meal_type != \"All\":",
    "            query += \" AND fl.Meal_Type = ?\"",
    "            params.append(selected_meal_type)",
    "",
    "        if search_keyword:",
    "            query += \" AND (LOWER(fl.Food_Name) LIKE ? OR LOWER(fl.Location) LIKE ?)\"",
    "            params.extend([f'%{search_keyword}%', f'%{search_keyword}%'])",
    "",
    "        query += \" AND date(fl.Expiry_Date) BETWEEN date(?) AND date(?)\"",
    "        params.append(expiry_start_date.strftime('%Y-%m-%d'))",
    "        params.append(expiry_end_date.strftime('%Y-%m-%d'))",
    "",
    "        query += \" ORDER BY fl.Expiry_Date ASC\"",
    "",
    "    filtered_listings = run_query(query, params)",
    "",
    "    def highlight_expiry_bold(s):",
    "        today = datetime.now().date()",
    "        style_string = [''] * len(s) ",
    "        ",
    "        if 'Expiry_Date' in s.index and not pd.isna(s['Expiry_Date']):",
    "            expiry_date_val = s['Expiry_Date'].date() ",
    "            ",
    "            if expiry_date_val < today: ",
    "                style_string = ['background-color: #ffe6e6; color: #8B0000; font-weight: bold;'] * len(s) ",
    "            elif (expiry_date_val - today).days <= 3: ",
    "                style_string = ['background-color: #fffacd; color: #cc6600; font-weight: bold;'] * len(s) ",
    "        ",
    "        return style_string",
    "",
    "    if not filtered_listings.empty:",
    "        filtered_listings['Expiry_Date'] = pd.to_datetime(filtered_listings['Expiry_Date'])",
    "        st.dataframe(filtered_listings.style.apply(highlight_expiry_bold, axis=1), use_container_width=True)",
    "        st.download_button(label='Download Filtered Listings (CSV) 📥', data=convert_df_to_csv(filtered_listings), file_name='filtered_food_listings.csv', mime='text/csv')",
    "        st.markdown('*(Expired listings are highlighted in <span style=\"background-color: #ffe6e6; color: #8B0000; font-weight: bold;\">light red & bold</span>; expiring within 3 days in <span style=\"background-color: #fffacd; color: #cc6600; font-weight: bold;\">light orange & bold</span>.)*', unsafe_allow_html=True)",
    "    else:",
    "        st.info(\"No food listings match your criteria.\")",
    "",
    "    st.subheader(\"SQL Query Used:\")",
    "    st.code(format_sql_query_for_display(query, params))",
    "",
    "elif page_selection == \"Claims & Status\":",
    "    st.header(\"Food Claims and Their Status \💬\")",
    "",
    "    if st.button('Refresh Claims Data 🔄', key='refresh_claims_data'):",
    "        st.rerun()",
    "",
    "    st.sidebar.subheader(\"Claim CRUD\")",
    "    claim_operation_selection = st.sidebar.radio(\"Choose Claim Action\", [\"View Claims\", \"Create New Claim\", \"Update Claim Status\", \"Delete Claim\"], help=\"Select a claim operation to perform.\")",
    "",
    "    if claim_operation_selection == \"View Claims\":",
    "        st.subheader(\"View All Claims \📋\")",
    "        col_cs_filter, col_cs_content = st.columns([1, 3])",
    "",
    "        with col_cs_filter:",
    "            st.subheader(\"Filter Options\")",
    "            claim_status_filter = st.selectbox(\"Filter by Status\", [\"All\", \"Completed\", \"Pending\", \"Cancelled\"], help=\"Filter claims based on their processing status.\", key='claims_filter_view')",
    "            st.write('---')",
    "",
    "        with col_cs_content:",
    "            st.subheader(\"Filtered Claims Data\")",
    "",
    "            query_claims = \"\"\"",
    "            SELECT",
    "                c.Claim_ID,",
    "                fl.Food_Name,",
    "                fl.Quantity,",
    "                p.Name AS Provider_Name,",
    "                r.Name AS Receiver_Name,",
    "                r.Contact AS Receiver_Contact,",
    "                c.Status,",
    "                c.Timestamp,",
    "                fl.Provider_ID AS Hidden_Provider_ID, r.Receiver_ID AS Hidden_Receiver_ID",
    "            FROM",
    "                Claims c",
    "            JOIN",
    "                Food_Listings fl ON c.Food_ID = fl.Food_ID",
    "            JOIN",
    "                Providers p ON fl.Provider_ID = p.Provider_ID",
    "            JOIN",
    "                Receivers r ON c.Receiver_ID = r.Receiver_ID",
    "            \"\"\"",
    "            params_claims = []",
    "",
    "            if claim_status_filter != \"All\":",
    "                query_claims += \" WHERE c.Status = ?\"",
    "                params_claims.append(claim_status_filter)",
    "",
    "            query_claims += \" ORDER BY c.Timestamp DESC\"",
    "",
    "            claims_data = run_query(query_claims, params_claims)",
    "",
    "            if not claims_data.empty:",
    "                claims_data_display = claims_data.drop(columns=['Hidden_Provider_ID', 'Hidden_Receiver_ID'], errors='ignore') ",
    "                ",
    "                claims_table_config = {",
    "                    claims_data_display.index.name if claims_data_display.index.name is not None else '_index': st.column_config.Column(",
    "                        \"Select Row\", ",
    "                        width='small',",
    "                        disabled=True",
    "                    ),",
    "                    'Status': st.column_config.Column(",
    "                         \"Status\",",
    "                         help=\"Current status of the claim\",",
    "                     )",
    "                }",
    "",
    "                selected_row_editor = st.data_editor(",
    "                    claims_data_display,",
    "                    num_rows=\"static\", ",
    "                    key='claims_data_editor_st_selection_view', ",
    "                    hide_index=False, ",
    "                    column_config=claims_table_config, ",
    "                    on_change=lambda: st.session_state.update(selected_claim_indices_key_view=st.session_state.claims_data_editor_st_selection_view['selection']['rows'])",
    "                )",
    "                ",
    "                selected_rows_indices = st.session_state.get('selected_claim_indices_key_view', [])",
    "",
    "                if selected_rows_indices:",
    "                    selected_original_df_index = selected_rows_indices[0]",
    "                    if selected_original_df_index < len(claims_data): ",
    "                        selected_row_data = claims_data.loc[selected_original_df_index]",
    "                        ",
    "                        provider_id = safe_int_conversion(selected_row_data['Hidden_Provider_ID'])",
    "                        receiver_id = safe_int_conversion(selected_row_data['Hidden_Receiver_ID'])",
    "",
    "                        st.markdown(\"--- Associated Entity Details ---\")",
    "                        col_pr_details1, col_pr_details2 = st.columns(2)",
    "                        with col_pr_details1:",
    "                            if provider_id is not None:",
    "                                provider_info_df = run_query(\"SELECT Name, Type, City, Contact FROM Providers WHERE Provider_ID = ?\", (provider_id,))",
    "                                if not provider_info_df.empty:",
    "                                    st.subheader(f\"Provider Profile (ID: {provider_id})\")",
    "                                    st.json(provider_info_df.to_dict('records')[0])",
    "                            else:",
    "                                st.info(\"Provider details not available.\")",
    "                        with col_pr_details2:",
    "                            if receiver_id is not None:",
    "                                receiver_info_df = run_query(\"SELECT Name, Type, City, Contact FROM Receivers WHERE Receiver_ID = ?\", (receiver_id,))",
    "                                if not receiver_info_df.empty:",
    "                                    st.subheader(f\"Receiver Profile (ID: {receiver_id})\")",
    "                                    st.json(receiver_info_df.to_dict('records')[0])",
    "                            else:",
    "                                st.info(\"Receiver details not available.\")",
    "                    else: ",
    "                        st.session_state.selected_claim_indices_key_view = [] ",
    "                        st.rerun() ",
    "                else: ",
    "                    st.info(\"Select a row from the table above to view associated Provider and Receiver details. Click a row to activate. 👆\")",
    "            else:",
    "                st.info(\"No claims data available based on current filters. 😟\")",
    "",
    "            st.subheader(\"SQL Query Used:\")",
    "            st.code(format_sql_query_for_display(query_claims, params_claims))",
    "",
    "            st.subheader(\"Claim Statistics\")",
    "            col1, col2, col3 = st.columns(3)",
    "            with col1:",
    "                st.metric(\"Total Claims\", claims_data.shape[0])",
    "            with col2:",
    "                st.metric(\"Completed Claims\", claims_data[claims_data['Status'] == 'Completed'].shape[0])",
    "            with col3:",
    "                st.metric(\"Pending Claims\", claims_data[claims_data['Status'] == 'Pending'].shape[0])",
    "",
    "    elif claim_operation_selection == \"Create New Claim\":",
    "        st.subheader(\"✨ Create a New Food Claim\")",
    "        with st.form(\"create_claim_form\"):",
    "            st.markdown(\"**Submit a claim for an available food item:**\")",
    "",
    "            max_claim_id_df = run_query(\"SELECT MAX(Claim_ID) FROM Claims\")",
    "            next_claim_id_suggestion = max_claim_id_df.iloc[0,0]",
    "            if pd.isna(next_claim_id_suggestion): next_claim_id_suggestion = 0",
    "            claim_id_for_creation = st.number_input(\"Claim ID (Must be unique integer)\", min_value=1, value=int(next_claim_id_suggestion) + 1, help=\"Provide a unique integer ID for this claim.\")",
    "",
    "            food_id_to_claim = None ",
    "            available_food_df = run_query(\"SELECT Food_ID, Food_Name, Quantity, Expiry_Date FROM Food_Listings WHERE Quantity > 0 ORDER BY Expiry_Date ASC\")",
    "            available_food_df['Expiry_Date'] = pd.to_datetime(available_food_df['Expiry_Date'], errors='coerce')",
    "            available_food_df = available_food_df[available_food_df['Expiry_Date'].notna()]",
    "",
    "            food_claim_display_options = []",
    "            if not available_food_df.empty: ",
    "                food_claim_display_options = [f\"{safe_int_conversion(row['Food_ID'])} - {row['Food_Name']} (Qty: {safe_int_conversion(row['Quantity'])}, Exp: {row['Expiry_Date'].strftime('%Y-%m-%d')})\" ",
    "                                          for idx, row in available_food_df.iterrows() if safe_int_conversion(row['Food_ID']) is not None]",
    "",
    "            food_selectbox_disabled = not bool(food_claim_display_options)",
    "            food_selectbox_options = food_claim_display_options if food_claim_display_options else ['(No food available)']",
    "            selected_food_claim_str = st.selectbox(\"Select Food Item to Claim\", food_selectbox_options, disabled=food_selectbox_disabled, key='food_claim_select')",
    "            if selected_food_claim_str and not food_selectbox_disabled and selected_food_claim_str != '(No food available)': ",
    "                food_id_to_claim = int(selected_food_claim_str.split(' - ')[0])",
    "            else:",
    "                food_id_to_claim = None",
    "",
    "            receiver_id_for_claim = None",
    "            receivers_df = run_query(\"SELECT Receiver_ID, Name, Type FROM Receivers WHERE Receiver_ID IS NOT NULL AND Name IS NOT NULL\")",
    "            receiver_claim_display_options = [f\"{safe_int_conversion(row['Receiver_ID'])} - {row['Name']} ({row['Type']})\" ",
    "                                              for idx, row in receivers_df.iterrows() if safe_int_conversion(row['Receiver_ID']) is not None]",
    "            ",
    "            receiver_selectbox_disabled = not bool(receiver_claim_display_options)",
    "            receiver_selectbox_options = receiver_claim_display_options if receiver_claim_display_options else ['(No receivers available)']",
    "            selected_receiver_claim_str = st.selectbox(\"Select Receiver\", receiver_selectbox_options, disabled=receiver_selectbox_disabled, key='receiver_claim_select')",
    "            if selected_receiver_claim_str and not receiver_selectbox_disabled and selected_receiver_claim_str != '(No receivers available)': ",
    "                receiver_id_for_claim = int(selected_receiver_claim_str.split(' - ')[0])",
    "            else:",
    "                receiver_id_for_claim = None",
    "",
    "            can_submit_claim = (food_id_to_claim is not None and receiver_id_for_claim is not None)",
    "",
    "            submitted = st.form_submit_button(\"Create Claim ✅\", disabled=not can_submit_claim)",
    "",
    "            if submitted:",
    "                if run_query(\"SELECT Claim_ID FROM Claims WHERE Claim_ID = ?\", (claim_id_for_creation,)).shape[0] > 0:",
    "                    st.error(f\"Claim ID {claim_id_for_creation} already exists. Please choose a different ID for the claim. ⛔\")",
    "                else:",
    "                    current_claim_status = \"Pending\" ",
    "                    current_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')",
    "                    success = create_claim(claim_id_for_creation, food_id_to_claim, receiver_id_for_claim, current_claim_status, current_timestamp)",
    "                    if success: st.rerun()",
    "            elif not can_submit_claim: ",
    "                st.info(\"Select a food item and a receiver to enable claim submission. 👆\")",
    "",
    "    elif claim_operation_selection == \"Update Claim Status\":",
    "        st.subheader(\"📝 Update an Existing Claim's Status\")",
    "        with st.form(\"update_claim_status_form\"):",
    "            all_claims_df = run_query(\"SELECT Claim_ID, Food_ID, Receiver_ID, Status, Timestamp FROM Claims ORDER BY Claim_ID DESC\")",
    "            if all_claims_df.empty:",
    "                st.info(\"No claims found to update. Create a claim first. 📝\")",
    "                st.stop()",
    "",
    "            claims_options = []",
    "            for idx, row in all_claims_df.iterrows():",
    "                claims_options.append(f\"{safe_int_conversion(row['Claim_ID'])} - Food: {safe_int_conversion(row['Food_ID'])}, Rec: {safe_int_conversion(row['Receiver_ID'])} (Status: {row['Status']})\")",
    "",
    "            selected_claim_str = st.selectbox(\"Select Claim to Update\", claims_options, help=\"Choose the claim record to modify its status.\", key='claims_update_select')",
    "            claim_id_to_update = int(selected_claim_str.split(' - ')[0])",
    "",
    "            current_status = all_claims_df[all_claims_df['Claim_ID'] == claim_id_to_update]['Status'].iloc[0]",
    "",
    "            new_status = st.selectbox(\"New Status\", [\"Pending\", \"Completed\", \"Cancelled\"], index=[\"Pending\", \"Completed\", \"Cancelled\"].index(current_status), help=\"Select the new status for this claim.\", key='claims_new_status_select')",
    "",
    "            update_submitted = st.form_submit_button(\"Update Claim Status ✅\")",
    "            if update_submitted:",
    "                success = update_claim_status(claim_id_to_update, new_status, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))",
    "                if success: st.rerun()",
    "",
    "    elif claim_operation_selection == \"Delete Claim\":",
    "        st.subheader(\"🗑️ Delete a Claim\")",
    "        with st.form(\"delete_claim_form\"):",
    "            all_claims_df = run_query(\"SELECT Claim_ID, Food_ID, Receiver_ID, Status, Timestamp FROM Claims ORDER BY Claim_ID DESC\")",
    "            if all_claims_df.empty:",
    "                st.info(\"No claims found to delete.\")",
    "                st.stop()",
    "",
    "            claims_options_delete = []",
    "            for idx, row in all_claims_df.iterrows():",
    "                claims_options_delete.append(f\"{safe_int_conversion(row['Claim_ID'])} - Food: {safe_int_conversion(row['Food_ID'])}, Rec: {safe_int_conversion(row['Receiver_ID'])} (Status: {row['Status']})\")",
    "",
    "            selected_claim_str_delete = st.selectbox(\"Select Claim to Delete\", claims_options_delete, help=\"Choose the claim record to delete.\", key='claims_delete_select')",
    "            claim_id_to_delete = int(selected_claim_str_delete.split(' - ')[0])",
    "",
    "            delete_submitted = st.form_submit_button(\"Delete Claim 🗑️\")",
    "            if delete_submitted:",
    "                st.warning(f\"Are you sure you want to delete Claim ID **{claim_id_to_delete}**? This action cannot be undone. ⚠️\")",
    "                if st.button(\"Confirm Permanent Delete\", key=\"confirm_delete_claim_btn\"):",
    "                    success = delete_claim(claim_id_to_delete)",
    "                    if success: st.rerun()",
    "",
    "    st.write('---')",
    "    st.subheader(\"Current Claims Overview \📋\")",
    "    all_current_claims = run_query(\"SELECT Claim_ID, Food_ID, Receiver_ID, Status, Timestamp FROM Claims ORDER BY Claim_ID DESC\")",
    "    st.dataframe(all_current_claims, use_container_width=True)",
    "    st.download_button(label='Download All Claims (CSV) 📥', data=convert_df_to_csv(all_current_claims), file_name='all_claims.csv', mime='text/csv')",
    "",
    "elif page_selection == \"At-Risk Food\":",
    "    st.header(\"At-Risk Food (Expiring Soon/Expired & Unclaimed) \🗑️\")",
    "    st.markdown(\"This section highlights food items that are expired or are due to expire very soon, and haven't been successfully claimed yet. Consider claiming these items urgently to prevent food waste!\")",
    "",
    "    if st.button('Refresh At-Risk Food 🔄', key='refresh_at_risk_food'):",
    "        st.rerun()",
    "",
    "    current_time_for_risk_filter = datetime.now().strftime('%Y-%m-%d %H:%M:%S')",
    "    future_risk_cutoff = (datetime.now() + timedelta(days=7)).strftime('%Y-%m-%d %H:%M:%S')",
    "",
    "    at_risk_food_query = f\"\"\"",
    "    SELECT",
    "        fl.Food_ID, fl.Food_Name, fl.Quantity, fl.Expiry_Date, p.Name AS Provider_Name, fl.Location, p.Contact AS Provider_Contact",
    "    FROM",
    "        Food_Listings fl",
    "    JOIN",
    "        Providers p ON fl.Provider_ID = p.Provider_ID",
    "    WHERE",
    "        fl.Quantity > 0",
    "        AND (date(fl.Expiry_Date) <= date('{current_time_for_risk_filter}') OR date(fl.Expiry_Date) BETWEEN date('{current_time_for_risk_filter}') AND date('{future_risk_cutoff}'))",
    "        AND NOT EXISTS (SELECT 1 FROM Claims c WHERE c.Food_ID = fl.Food_ID AND c.Status IN ('Pending', 'Completed'))",
    "    ORDER BY",
    "        fl.Expiry_Date ASC;",
    "    \"\"\"",
    "    at_risk_df = run_query(at_risk_food_query)",
    "",
    "    if not at_risk_df.empty:",
    "        at_risk_df['Expiry_Date'] = pd.to_datetime(at_risk_df['Expiry_Date'], errors='coerce')",
    "        at_risk_df = at_risk_df[at_risk_df['Expiry_Date'].notna()] ",
    "        st.dataframe(at_risk_df.style.apply(highlight_expiry_bold, axis=1), use_container_width=True)",
    "        st.markdown('*(Expired items and those expiring within the next 7 days, with quantity > 0 and no current pending/completed claims, are shown here.)*')",
    "        st.download_button(label='Download At-Risk Food (CSV) 📥', data=convert_df_to_csv(at_risk_df), file_name='at_risk_food.csv', mime='text/csv')",
    "",
    "        st.write('---')",
    "        st.subheader(\"Claim an At-Risk Food Item \🎁\")",
    "        st.markdown(\"You can directly claim one of the above at-risk food items.\")",
    "        with st.form(\"claim_at_risk_food_form\"):",
    "            at_risk_food_options = [f\"{safe_int_conversion(row['Food_ID'])} - {row['Food_Name']} (Qty: {safe_int_conversion(row['Quantity'])}, Exp: {row['Expiry_Date'].strftime('%Y-%m-%d')})\" ",
    "                                    for idx, row in at_risk_df.iterrows() if safe_int_conversion(row['Food_ID']) is not None]",
    "",
    "            food_id_to_claim_at_risk = None ",
    "            at_risk_food_selectbox_disabled = not bool(at_risk_food_options)",
    "            at_risk_food_selectbox_options = at_risk_food_options if at_risk_food_options else ['(No At-Risk food available)']",
    "            selected_at_risk_food_str = st.selectbox(\"Select At-Risk Food to Claim\", at_risk_food_selectbox_options, disabled=at_risk_food_selectbox_disabled, key='at_risk_food_select')",
    "            if selected_at_risk_food_str and not at_risk_food_selectbox_disabled and selected_at_risk_food_str != '(No At-Risk food available)':",
    "                food_id_to_claim_at_risk = int(selected_at_risk_food_str.split(' - ')[0])",
    "",
    "            receivers_df_at_risk = run_query(\"SELECT Receiver_ID, Name, Type FROM Receivers WHERE Receiver_ID IS NOT NULL AND Name IS NOT NULL\")",
    "            receiver_at_risk_display_options = [f\"{safe_int_conversion(row['Receiver_ID'])} - {row['Name']} ({row['Type']})\" ",
    "                                              for idx, row in receivers_df_at_risk.iterrows() if safe_int_conversion(row['Receiver_ID']) is not None]",
    "            ",
    "            receiver_at_risk_selectbox_disabled = not bool(receiver_at_risk_display_options)",
    "            receiver_at_risk_selectbox_options = receiver_at_risk_display_options if receiver_at_risk_display_options else ['(No receivers available)']",
    "            selected_receiver_at_risk_str = st.selectbox(\"Select Receiver for At-Risk Food\", receiver_at_risk_selectbox_options, disabled=receiver_at_risk_selectbox_disabled, key='at_risk_receiver_claim_select')",
    "            receiver_id_for_claim_at_risk = None",
    "            if selected_receiver_at_risk_str and not receiver_at_risk_selectbox_disabled and selected_receiver_at_risk_str != '(No receivers available)':",
    "                receiver_id_for_claim_at_risk = int(selected_receiver_at_risk_str.split(' - ')[0])",
    "",
    "            max_claim_id_df_at_risk = run_query(\"SELECT MAX(Claim_ID) FROM Claims\")",
    "            next_claim_id_suggestion_at_risk = max_claim_id_df_at_risk.iloc[0,0]",
    "            if pd.isna(next_claim_id_suggestion_at_risk): next_claim_id_suggestion_at_risk = 0",
    "            claim_id_for_creation_at_risk = st.number_input(\"Claim ID (For At-Risk Food Claim)\", min_value=1, value=int(next_claim_id_suggestion_at_risk) + 1, help=\"Provide a unique integer ID for this claim.\", key='at_risk_claim_id')",
    "",
    "            can_submit_at_risk_claim = (food_id_to_claim_at_risk is not None and receiver_id_for_claim_at_risk is not None)",
    "",
    "            submitted_at_risk_claim = st.form_submit_button(\"Claim At-Risk Food ✅\", disabled=not can_submit_at_risk_claim)",
    "",
    "            if submitted_at_risk_claim:",
    "                if run_query(\"SELECT Claim_ID FROM Claims WHERE Claim_ID = ?\", (claim_id_for_creation_at_risk,)).shape[0] > 0:",
    "                    st.error(f\"Claim ID {claim_id_for_creation_at_risk} already exists. Please choose a different ID for the claim. ⛔\")",
    "                else:",
    "                    current_timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')",
    "                    success = create_claim(claim_id_for_creation_at_risk, food_id_to_claim_at_risk, receiver_id_for_claim_at_risk, \"Pending\", current_timestamp)",
    "                    if success: st.rerun()",
    "            elif not can_submit_at_risk_claim: ",
    "                 st.info(\"Select an at-risk food item and a receiver to enable claiming. 👆\")",
    "    else:",
    "        st.info(\"No food items are currently classified as 'At-Risk' (Expired or Expiring Soon, and Unclaimed). Good job! 🎉\")",
    "",
    "elif page_selection == \"CRUD Operations\":",
    "    st.header(\"Manage Food Listings (CRUD) \📝\")",
    "    st.warning(\"Exercise caution with these operations as they modify the database directly. Deleting a food listing will also remove associated claims. 🛑\", icon=\"⚠️\")",
    "",
    "    if st.button('Refresh Current Listings 🔄', key='refresh_crud_data', help=\"Click to reload the table and forms after manual database changes.\"):",
    "        st.rerun()",
    "",
    "    crud_action = st.sidebar.radio(\"Choose Action\", [\"Add New Listing\", \"Update Existing Listing\", \"Delete Listing\"], help=\"Select a CRUD operation to perform.\")",
    "",
    "    if crud_action == \"Add New Listing\":",
    "        st.subheader(\"Add New Food Listing \➕\")",
    "        with st.form(\"add_listing_form\"):",
    "            max_id_df = run_query(\"SELECT MAX(Food_ID) FROM Food_Listings\")",
    "            next_id_suggestion = max_id_df.iloc[0,0]",
    "            if pd.isna(next_id_suggestion): next_id_suggestion = 0",
    "            ",
    "            food_id = st.number_input(\"Food ID (Must be unique integer)\", min_value=1, value=int(next_id_suggestion) + 1, step=1, help=\"Provide a unique integer ID for this food item.\")",
    "            ",
    "            food_name = st.text_input(\"Food Name\", help=\"e.g., Bread, Chicken Salad, Fresh Fruits\")",
    "            quantity = st.number_input(\"Quantity\", min_value=1, help=\"Number of units, kg, or pieces.\")",
    "            default_expiry = datetime.now().date() + timedelta(days=365)",
    "            expiry_date = st.date_input(\"Expiry Date\", value=default_expiry, min_value=datetime.now().date(), help=\"Select the expiry date for the food item.\")",
    "",
    "            existing_providers = run_query(\"SELECT Provider_ID, Name, Type FROM Providers WHERE Provider_ID IS NOT NULL AND Name IS NOT NULL AND Type IS NOT NULL\")",
    "            provider_options_display = [f\"{safe_int_conversion(row['Provider_ID'])} - {row['Name']} ({row['Type']})\" for idx, row in existing_providers.iterrows() if safe_int_conversion(row['Provider_ID']) is not None]",
    "            if not provider_options_display:",
    "                st.error(\"No valid providers found in the database. Please add providers first (or ensure data is loaded correctly). 🛑\")",
    "                st.stop()",
    "",
    "            selected_provider_display = st.selectbox(\"Select Provider\", provider_options_display, help=\"Choose an existing provider from whom this food originates.\")",
    "            ",
    "            selected_provider_id_num = int(selected_provider_display.split(' - ')[0])",
    "            provider_id_for_insert = selected_provider_id_num",
    "            provider_type_for_insert = existing_providers[existing_providers['Provider_ID'] == provider_id_for_insert]['Type'].iloc[0]",
    "",
    "            all_provider_cities = run_query(\"SELECT DISTINCT City FROM Providers WHERE City IS NOT NULL\").iloc[:,0].tolist()",
    "            if not all_provider_cities: all_provider_cities = ['(N/A)']",
    "            location = st.selectbox(\"Food Location (Provider's City)\", all_provider_cities, help=\"Location should generally be the provider's city for easy identification.\")",
    "",
    "            food_type = st.selectbox(\"Food Type\", [\"Vegetarian\", \"Non-Vegetarian\", \"Vegan\"])",
    "            meal_type = st.selectbox(\"Meal Type\", [\"Breakfast\", \"Lunch\", \"Dinner\", \"Snacks\"])",
    "",
    "            submitted = st.form_submit_button(\"Add Listing ✅\")",
    "            if submitted:",
    "                if run_query(\"SELECT Food_ID FROM Food_Listings WHERE Food_ID = ?\", (food_id,)).shape[0] > 0: ",
    "                    st.error(f\"Food ID {food_id} already exists. Please choose a different ID. ⛔\")",
    "                else:",
    "                    add_food_listing(food_id, food_name, quantity, expiry_date.strftime('%Y-%m-%d'), provider_id_for_insert, provider_type_for_insert, location, food_type, meal_type)",
    "                    st.rerun()",
    "",
    "    elif crud_action == \"Update Existing Listing\":",
    "        st.subheader(\"Update Existing Food Listing \🖊️\")",
    "        all_food_listings_data = run_query(\"SELECT Food_ID, Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location FROM Food_Listings\")",
    "        valid_listings = all_food_listings_data[all_food_listings_data['Food_ID'].notna() & all_food_listings_data['Food_Name'].notna()]",
    "        all_food_ids_options = [f\"{safe_int_conversion(row['Food_ID'])} - {row['Food_Name']}\" for idx, row in valid_listings.iterrows()]",
    "        ",
    "        if not all_food_ids_options:",
    "            st.info(\"No valid food listings found to update. Add a listing first. 📝\")",
    "            st.stop()",
    "",
    "        selected_food_id_str = st.selectbox(\"Select Food Listing to Update (ID - Name)\", all_food_ids_options, help=\"Choose the food item you wish to modify. Details will load below.\")",
    "",
    "        if selected_food_id_str:",
    "            food_id_to_update = int(selected_food_id_str.split(' - ')[0])",
    "            current_data = run_query(\"SELECT Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location FROM Food_Listings WHERE Food_ID = ?\", (food_id_to_update,)).iloc[0]",
    "",
    "            with st.form(\"update_listing_form\"):",
    "                st.markdown(f\"### Updating Food ID: **{food_id_to_update}** | Name: **{current_data['Food_Name']}**\")",
    "                st.text_input(\"Food Name (Non-editable)\", value=current_data['Food_Name'], disabled=True, help=\"Food name cannot be changed directly from here. For complex changes, delete and re-add.\")",
    "                st.text_input(\"Current Provider ID (Non-editable)\", value=safe_int_conversion(current_data['Provider_ID']), disabled=True, help=\"Provider details are fixed once listed. \")",
    "                st.text_input(\"Current Provider Type (Non-editable)\", value=current_data['Provider_Type'], disabled=True, help=\"This is derived from the Provider ID and cannot be changed here.\")",
    "                st.text_input(\"Current Location (Non-editable)\", value=current_data['Location'], disabled=True, help=\"Location is tied to the provider and cannot be changed here.\")",
    "",
    "                new_quantity = st.number_input(\"New Quantity\", min_value=1, value=safe_int_conversion(current_data['Quantity']), help=\"Update the available quantity of this food item.\")",
    "                try:",
    "                    current_expiry_date_obj = datetime.strptime(str(current_data['Expiry_Date']), '%Y-%m-%d %H:%M:%S').date()",
    "                except ValueError:",
    "                    current_expiry_date_obj = datetime.now().date()",
    "                new_expiry_date = st.date_input(\"New Expiry Date\", value=current_expiry_date_obj, help=\"Update the expiry date for this food item.\")",
    "",
    "                submitted = st.form_submit_button(\"Update Listing ✅\")",
    "                if submitted:",
    "                    update_food_listing(food_id_to_update, new_quantity, new_expiry_date.strftime('%Y-%m-%d'))",
    "                    st.rerun()",
    "",
    "    elif crud_action == \"Delete Listing\":",
    "        st.subheader(\"Delete Food Listing \✖️\")",
    "        all_food_listings_data = run_query(\"SELECT Food_ID, Food_Name FROM Food_Listings\")",
    "        valid_listings = all_food_listings_data[all_food_listings_data['Food_ID'].notna() & all_food_listings_data['Food_Name'].notna()]",
    "        all_food_ids_options = [f\"{safe_int_conversion(row['Food_ID'])} - {row['Food_Name']}\" for idx, row in valid_listings.iterrows()]",
    "",
    "        if not all_food_ids_options:",
    "            st.info(\"No valid food listings found to delete. 📝\")",
    "            st.stop()",
    "",
    "        selected_food_id_str = st.selectbox(\"Select Food Listing to Delete (ID - Name)\", all_food_ids_options, help=\"Choose the food item to be permanently removed.\")",
    "",
    "        if selected_food_id_str:",
    "            food_id_to_delete = int(selected_food_id_str.split(' - ')[0])",
    "            st.warning(f\"Are you sure you want to delete listing ID **{food_id_to_delete}** (\\\"{selected_food_id_str.split(' - ')[1]}\\\")? This will also delete any associated claims. ⚠️\")",
    "            if st.button(\"Confirm Delete 🗑️\"):",
    "                delete_food_listing(food_id_to_delete)",
    "                st.rerun()",
    "",
    "    st.subheader(\"Current Food Listings Overview \📋\")",
    "    all_current_listings = run_query(\"SELECT * FROM Food_Listings\")",
    "    st.dataframe(all_current_listings, use_container_width=True)",
    "    st.download_button(label='Download All Food Listings (CSV) 📥', data=convert_df_to_csv(all_current_listings), file_name='all_food_listings.csv', mime='text/csv')",
    "",
    "st.write(\"--- End of Application --- 🎉\")"
]

with open('app.py', 'w') as f:
    for line in streamlit_app_lines:
        f.write(line + "\n")

print("\nStreamlit app script 'app.py' created with definitive string escaping.")
print("Remember to re-run Cell 3.2 to deploy the updated app.")


Streamlit app script 'app.py' created with definitive string escaping.
Remember to re-run Cell 3.2 to deploy the updated app.


In [165]:
# Cell to kill all ngrok processes
!killall ngrok

In [166]:
# Set up ngrok tunnel
from pyngrok import ngrok # Import ngrok

ngrok.set_auth_token("31Bv4nMZjAhTl39LxWjG0r0eTbA_4ucCMfDuhbzMTgTqik7YX")  # Get from https://dashboard.ngrok.com/
public_url = ngrok.connect(8501).public_url
print(f"Streamlit app running at: {public_url}")

# Run Streamlit in background
!streamlit run app.py --server.port 8501 &>/dev/null&

Streamlit app running at: https://657efe851f87.ngrok-free.app
